In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Cargar el archivo CSV
file_path = 'FuelConsumptionCo2.csv'
data = pd.read_csv(file_path)

# Mostrar las primeras filas del dataset
data.head()

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2022,Acura,ILX,Compact,2.4,4,AM8,Z,9.9,7.0,8.6,33,200
1,2022,Acura,MDX SH-AWD,SUV: Small,3.5,6,AS10,Z,12.6,9.4,11.2,25,263
2,2022,Acura,RDX SH-AWD,SUV: Small,2.0,4,AS10,Z,11.0,8.6,9.9,29,232
3,2022,Acura,RDX SH-AWD A-SPEC,SUV: Small,2.0,4,AS10,Z,11.3,9.1,10.3,27,242
4,2022,Acura,TLX SH-AWD,Compact,2.0,4,AS10,Z,11.2,8.0,9.8,29,230


## Elimine las columnas de tipo categórico de la base de datos y verifique que no existan datos nulos que deban ser eliminados de manera previa al análisis.

In [3]:
# Eliminar columnas de tipo categórico
data = data.select_dtypes(exclude=['object'])

# Verificar si existen datos nulos
print("¿Existen datos nulos?")
print(data.isnull().sum())

# Eliminar filas con datos nulos si es necesario
data = data.dropna()

# Confirmar que no existan datos nulos después de la limpieza
print("¿Existen datos nulos después de la limpieza?")
print(data.isnull().sum())

# Mostrar las primeras filas del dataset limpio
data.head()

¿Existen datos nulos?
MODELYEAR                   0
ENGINESIZE                  0
CYLINDERS                   0
FUELCONSUMPTION_CITY        0
FUELCONSUMPTION_HWY         0
FUELCONSUMPTION_COMB        0
FUELCONSUMPTION_COMB_MPG    0
CO2EMISSIONS                0
dtype: int64
¿Existen datos nulos después de la limpieza?
MODELYEAR                   0
ENGINESIZE                  0
CYLINDERS                   0
FUELCONSUMPTION_CITY        0
FUELCONSUMPTION_HWY         0
FUELCONSUMPTION_COMB        0
FUELCONSUMPTION_COMB_MPG    0
CO2EMISSIONS                0
dtype: int64


,MODELYEAR,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2022,2.4,4,9.9,7.0,8.6,33,200
1,2022,3.5,6,12.6,9.4,11.2,25,263
2,2022,2.0,4,11.0,8.6,9.9,29,232
3,2022,2.0,4,11.3,9.1,10.3,27,242
4,2022,2.0,4,11.2,8.0,9.8,29,230


## Realice un análisis de regresión múltiple para pronosticar la variable “CO2 EMISSIONS” con las variables remanentes. Obtenga los indicadores de bondad de ajuste correspondientes (R cuadrada, Error medio absoluto, etc.). Utilice los coeficientes resultantes y pronostique la primera observación de la base de prueba. ¿Coincide su resultado con aquel obtenido con la instrucción “predict”? Explique.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Separar las variables predictoras (X) y la variable objetivo (y)
X = data.drop(columns=['CO2EMISSIONS'])
y = data['CO2EMISSIONS']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular indicadores de bondad de ajuste
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R cuadrada: {r2}")
print(f"Error medio absoluto (MAE): {mae}")
print(f"Raíz del error cuadrático medio (RMSE): {rmse}")

# Pronosticar la primera observación del conjunto de prueba
first_observation = X_test.iloc[0]
manual_prediction = np.dot(first_observation, model.coef_) + model.intercept_
predict_function_result = model.predict([first_observation])

print("\nPronóstico manual de la primera observación:", manual_prediction)
print("Resultado de la función predict:", predict_function_result[0])

# Comparar los resultados
if np.isclose(manual_prediction, predict_function_result[0]):
    print("\nEl pronóstico manual coincide con el resultado de la función 'predict'.")
else:
    print("\nEl pronóstico manual NO coincide con el resultado de la función 'predict'.")

R cuadrada: 0.9004924315633442
Error medio absoluto (MAE): 7.339144920646463
Raíz del error cuadrático medio (RMSE): 19.20591732975203

Pronóstico manual de la primera observación: 242.92694037393747
Resultado de la función predict: 242.92694037393747

El pronóstico manual coincide con el resultado de la función 'predict'.


## Repita el ejercicio 3 pero aplicando un modelo de regresión Ridge mediante el valor de Alpha óptimo.

In [5]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Separar las variables predictoras (X) y la variable objetivo (y)
X = data.drop(columns=['CO2EMISSIONS'])
y = data['CO2EMISSIONS']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo de regresión Ridge y los valores de alpha a probar
ridge = Ridge()
param_grid = {'alpha': np.logspace(-4, 4, 50)}  # Valores de alpha en escala logarítmica

# Usar GridSearchCV para encontrar el mejor valor de alpha
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo y el valor óptimo de alpha
best_ridge = grid_search.best_estimator_
best_alpha = grid_search.best_params_['alpha']
print(f"Mejor valor de alpha: {best_alpha}")

# Realizar predicciones en el conjunto de prueba
y_pred = best_ridge.predict(X_test)

# Calcular indicadores de bondad de ajuste
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R cuadrada: {r2}")
print(f"Error medio absoluto (MAE): {mae}")
print(f"Raíz del error cuadrático medio (RMSE): {rmse}")

# Pronosticar la primera observación del conjunto de prueba
first_observation = X_test.iloc[0]
manual_prediction = np.dot(first_observation, best_ridge.coef_) + best_ridge.intercept_
predict_function_result = best_ridge.predict([first_observation])

print("\nPronóstico manual de la primera observación:", manual_prediction)
print("Resultado de la función predict:", predict_function_result[0])

# Comparar los resultados
if np.isclose(manual_prediction, predict_function_result[0]):
    print("\nEl pronóstico manual coincide con el resultado de la función 'predict'.")
else:
    print("\nEl pronóstico manual NO coincide con el resultado de la función 'predict'.")

Mejor valor de alpha: 75.43120063354607
R cuadrada: 0.9030866805556986
Error medio absoluto (MAE): 7.385792101557064
Raíz del error cuadrático medio (RMSE): 18.95390644875932

Pronóstico manual de la primera observación: 243.83000180700537
Resultado de la función predict: 243.83000180700537

El pronóstico manual coincide con el resultado de la función 'predict'.


## Repita el ejercicio 3 pero aplicando un modelo de regresión Lasso mediante el valor de Alpha óptimo. 

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Separar las variables predictoras (X) y la variable objetivo (y)
X = data.drop(columns=['CO2EMISSIONS'])
y = data['CO2EMISSIONS']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo de regresión Ridge y los valores de alpha a probar
ridge = Ridge()
param_grid = {'alpha': np.logspace(-4, 4, 50)}  # Valores de alpha en escala logarítmica

# Usar GridSearchCV para encontrar el mejor valor de alpha
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo y el valor óptimo de alpha
best_ridge = grid_search.best_estimator_
best_alpha = grid_search.best_params_['alpha']
print(f"Mejor valor de alpha: {best_alpha}")

# Realizar predicciones en el conjunto de prueba
y_pred = best_ridge.predict(X_test)

# Calcular indicadores de bondad de ajuste
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R cuadrada: {r2}")
print(f"Error medio absoluto (MAE): {mae}")
print(f"Raíz del error cuadrático medio (RMSE): {rmse}")

# Pronosticar la primera observación del conjunto de prueba
first_observation = X_test.iloc[0]
manual_prediction = np.dot(first_observation, best_ridge.coef_) + best_ridge.intercept_
predict_function_result = best_ridge.predict([first_observation])

print("\nPronóstico manual de la primera observación:", manual_prediction)
print("Resultado de la función predict:", predict_function_result[0])

# Comparar los resultados
if np.isclose(manual_prediction, predict_function_result[0]):
    print("\nEl pronóstico manual coincide con el resultado de la función 'predict'.")
else:
    print("\nEl pronóstico manual NO coincide con el resultado de la función 'predict'.")

Mejor valor de alpha: 75.43120063354607
R cuadrada: 0.9030866805556986
Error medio absoluto (MAE): 7.385792101557064
Raíz del error cuadrático medio (RMSE): 18.95390644875932

Pronóstico manual de la primera observación: 243.83000180700537
Resultado de la función predict: 243.83000180700537

El pronóstico manual coincide con el resultado de la función 'predict'.


## Conclusion

 El mejor modelo es el Modelo 2 (Regresión Ridge con Alpha óptimo) o el Modelo 3 (Regresión Lasso con Alpha óptimo), ya que ambos tienen el mayor valor de R² (0.9031) y el menor RMSE (18.9539), lo que indica un mejor ajuste y mayor precisión en las predicciones en comparación con el Modelo 1 (Regresión Lineal).

Aunque el Modelo 3 (Lasso) y el Modelo 2 (Ridge) tienen el mismo desempeño en este caso, Ridge podría ser preferido si se busca evitar la eliminación de características, ya que Lasso tiende a reducir algunos coeficientes a cero, lo que puede ser útil para selección de variables pero no siempre deseable.

Por lo tanto, Modelo 2 (Ridge) es una opción ligeramente más robusta en este caso, pero ambos (Ridge y Lasso) son superiores al Modelo 1 (Regresión Lineal)